In [2]:
import sys
print(sys.executable)

/Users/madhang/Downloads/water_quality_project/waterenv/bin/python3


In [3]:
import tensorflow as tf
print(tf.__version__)

2.20.0


In [1]:
pip install pandas numpy scikit-learn tensorflow


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout

# -----------------------------
# 1. Load dataset
# -----------------------------
df = pd.read_csv("water_potability.csv")

# -----------------------------
# 2. Separate features & target
# -----------------------------
X = df.drop("Potability", axis=1)
y = df["Potability"]

# -----------------------------
# 3. Handle missing values
# -----------------------------
imputer = SimpleImputer(strategy="mean")
X = imputer.fit_transform(X)

# -----------------------------
# 4. Normalize
# -----------------------------
scaler = StandardScaler()
X = scaler.fit_transform(X)

# -----------------------------
# 5. Convert to time-series windows
# (important for CNN-LSTM)
# -----------------------------
WINDOW = 5   # 5 consecutive readings = 1 sample

X_seq = []
y_seq = []

for i in range(len(X) - WINDOW):
    X_seq.append(X[i:i+WINDOW])
    y_seq.append(y[i+WINDOW])

X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

# shape -> (samples, timesteps, features)
print(X_seq.shape)

# -----------------------------
# 6. Train test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_seq, y_seq, test_size=0.2, random_state=42
)

# -----------------------------
# 7. CNN-LSTM model
# -----------------------------
model = Sequential([
    Conv1D(64, 2, activation='relu', input_shape=(WINDOW, X_seq.shape[2])),
    MaxPooling1D(2),

    LSTM(64, return_sequences=False),

    Dropout(0.3),

    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# -----------------------------
# 8. Train
# -----------------------------
model.fit(
    X_train,
    y_train,
    epochs=40,
    batch_size=32,
    validation_split=0.2
)

# -----------------------------
# 9. Evaluate
# -----------------------------
loss, acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", acc)

# -----------------------------
# 10. Save model
# -----------------------------
model.save("cnn_lstm_water_model.keras")

(3271, 5, 9)
Epoch 1/40


/Users/madhang/Downloads/water_quality_project/waterenv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6147 - loss: 0.6590 - val_accuracy: 0.6851 - val_loss: 0.6232
Epoch 2/40
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7213 - loss: 0.5602 - val_accuracy: 0.6775 - val_loss: 0.6105
Epoch 3/40
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7725 - loss: 0.4856 - val_accuracy: 0.7748 - val_loss: 0.4857
Epoch 4/40
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8059 - loss: 0.4358 - val_accuracy: 0.8015 - val_loss: 0.4568
Epoch 5/40
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8327 - loss: 0.4015 - val_accuracy: 0.7958 - val_loss: 0.4677
Epoch 6/40
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8375 - loss: 0.3836 - val_accuracy: 0.8053 - val_loss: 0.4401
Epoch 7/40
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8480 - loss: 0.3597 - val_accuracy: 0.8111 - val_loss: 0.4317
Epoch 8/40
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8599 - loss: 0.3346 - val_accuracy: 0.8015 - val_loss: 0.4345
Epo

In [6]:
import numpy as np
import tensorflow as tf
import joblib

WINDOW = 5

model = tf.keras.models.load_model("cnn_lstm_water_model.keras")
scaler = joblib.load("scaler.pkl")
imputer = joblib.load("imputer.pkl")

# unsafe time-series (varying values)
sample = np.array([
    [5.3,355,33000,12,530,970,28,160,9],
    [5.2,360,34000,13,540,980,29,170,9.5],
    [5.1,370,35000,14,560,990,30,175,10],
    [5.0,380,36000,15,580,1000,31,180,10.5],
    [4.9,390,37000,16,600,1010,32,185,11]
])

sample = imputer.transform(sample)
sample = scaler.transform(sample)
sample = sample.reshape(1, WINDOW, 9)

p = float(model.predict(sample, verbose=0)[0][0])

status = "SAFE" if p > 0.7 else "MODERATE" if p > 0.3 else "UNSAFE"

print("Probability:", p)
print("Status:", status)

Probability: 0.999997079372406
Status: SAFE


/Users/madhang/Downloads/water_quality_project/waterenv/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


In [7]:
print("Scaled input:\n", sample.reshape(5,9))

Scaled input:
 [[-1.21164606  4.82530023  1.25306447  3.08162113  5.42999433  6.7291581
   4.14644854  5.93650611  6.45066079]
 [-1.27968569  4.97739267  1.36712555  3.71339562  5.70671828  6.85290251
   4.44877734  6.5707231   7.09147012]
 [-1.34772533  5.28157756  1.48118663  4.34517012  6.26016619  6.97664693
   4.75110615  6.8878316   7.73227945]
 [-1.41576496  5.58576245  1.59524771  4.97694462  6.8136141   7.10039135
   5.05343495  7.20494009  8.37308878]
 [-1.4838046   5.88994734  1.70930879  5.60871912  7.36706201  7.22413576
   5.35576376  7.52204858  9.01389812]]


In [17]:
import numpy as np
import tensorflow as tf

# load trained model
model = tf.keras.models.load_model("cnn_lstm_water_model.keras")

# -------------------------------------------------
# MANUAL POTABLE SENSOR READINGS (5 timesteps × 9)
# -------------------------------------------------

# sample = np.array([
#     [7.2,120,320,6.5,180,280,6.0,35,1.2],
#     [7.1,115,300,6.6,170,275,5.8,34,1.3],
#     [7.3,110,310,6.4,165,290,6.2,36,1.1],
#     [7.2,125,305,6.7,175,285,5.9,37,1.4],
#     [7.4,118,295,6.5,160,270,6.1,33,1.2]
# ])
#non potable 
sample = np.array([
    [5.5,350,30000,10,500,900,25,150,8],
    [5.6,360,32000,11,520,950,27,155,9],
    [5.4,340,31000,10,510,920,26,148,8],
    [5.3,355,33000,12,530,970,28,160,9],
    [5.5,345,31500,11,505,910,25,152,8]
])

# ---------- remove NaN if any ----------
col_mean = np.nanmean(sample, axis=0)
inds = np.where(np.isnan(sample))
sample[inds] = np.take(col_mean, inds[1])

# ---------- safe normalization ----------
mean = sample.mean(axis=0)
std = sample.std(axis=0)
std[std == 0] = 1

sample = (sample - mean) / std

# reshape for CNN-LSTM
sample = sample.reshape(1, 5, 9)

# predict
p = model.predict(sample)[0][0]

print("Probability:", p)
print("Predicted Class:", 1 if p > 0.5 else 0)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Probability: 0.10260988
Predicted Class: 0


In [5]:
import numpy as np
import tensorflow as tf
import joblib

WINDOW = 5

model = tf.keras.models.load_model("cnn_lstm_water_model.keras")
scaler = joblib.load("scaler.pkl")
imputer = joblib.load("imputer.pkl")

sample = np.array([
    [5.3,355,33000,12,530,970,28,160,9],
    [5.2,360,34000,13,540,980,29,170,9.5],
    [5.1,370,35000,14,560,990,30,175,10],
    [5.0,380,36000,15,580,1000,31,180,10.5],
    [4.9,390,37000,16,600,1010,32,185,11]
])

# repeat same row 5 times (same as backend window)
sample = np.array([reading] * WINDOW)

sample = imputer.transform(sample)
sample = scaler.transform(sample)

sample = sample.reshape(1, WINDOW, 9)

p = float(model.predict(sample, verbose=0)[0][0])

status = "SAFE" if p > 0.7 else "MODERATE" if p > 0.3 else "UNSAFE"

print("Probability:", p)
print("Status:", status)


Probability: 0.9999926090240479
Status: SAFE


/Users/madhang/Downloads/water_quality_project/waterenv/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
